In [72]:
from old_code.dataset import *
import tensorflow as tf
from MLP import *

In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
# Test training LinearModel on 2024-08-21 data
import pickle
import os
import numpy as np
from old_code.dataset import get_scaled_split_data
from MLP import MLP
from Model import Model
from Model import EnsembleModel
import pandas as pd
from Dataset import Dataset
from omegaconf import OmegaConf

cfg1 = OmegaConf.load("/usr/project/chembattery/chembattery_refac/configs/train_lfp_reproduce3.yaml")
cfg2 = OmegaConf.load("/usr/project/chembattery/chembattery_refac/configs/train_lfp_reproduce3_finetune.yaml")
cfgr = OmegaConf.load("/usr/project/chembattery/chembattery_refac/configs/train_lfp_reproduce3.yaml")

In [96]:
#IMPORTANT: code for editing config files. Do not delete
src_dir = "./configs/fine_tune_tests"
dest_dir = "./configs/fine_tune_tests_ensemble"
os.makedirs(dest_dir, exist_ok=True)
for f in os.listdir(src_dir):
    if "yaml" in f:
        src = os.path.join(src_dir, f)
        dest = os.path.join(dest_dir, f)
        config = OmegaConf.load(src)
        config.model.num_models = 5
        config.trainer.save_dir = "./runs/finetune_tests_ensemble"
        ckpt = config.trainer.get("ckpt_dir", None)
        if ckpt:
            new_ckpt = ckpt.split("/")
            new_ckpt = new_ckpt[:-2] + ["finetune_test_ensemble"] + [new_ckpt[-1]]
            new_ckpt = os.path.join(*new_ckpt)
            config.trainer.ckpt_dir = new_ckpt
            print(new_ckpt)
        config.trainer.epochs /= 5
        OmegaConf.save(config, dest)

In [98]:
src_dir = "./configs/fine_tune_tests_ensemble"
for f in os.listdir(src_dir):
    if "yaml" in f and "finetune" in f:
        src = os.path.join(src_dir, f)
        dest = os.path.join(src_dir, f)
        config = OmegaConf.load(src)
        config.trainer.learning_rate = 0.0003
        OmegaConf.save(config, dest)

In [85]:
">".join(["he", "lo"])

'he>lo'

In [99]:


data_path = os.path.join(cfg.data.data_dir, cfg.data.date_str)
scale_path = os.path.join(cfg.data.data_dir, cfg.data.get("scaler_date", '⚖️'))
features = cfg.data.get("features", None)
dataset = Dataset(data_path, scale_path, features)

trim_to = cfg.data.get("dataset_max", None)
if trim_to is not None:
    dataset.trim(int(trim_to))

prepros_steps = cfg.data.get("preprocessing", [])
print("[reprros: ]", prepros_steps)

print("splitting data")
split_on = cfg.data.get("split_on", "cell_num")
test_split = [str(id) for id in cfg.data.test_split.split(',')]
X_train, X_test, y_train, y_test, scaler = dataset.get_scaled_split(test_split, split_on)

[reprros: ] []
splitting data


In [104]:
dataset.masterList

cell_num
1        0.33
2        0.33
3        0.33
4        0.33
5         0.5
6         0.5
7         0.5
8         0.5
9         0.5
10        0.8
11        0.8
12        0.8
13        0.8
14        0.8
15        0.8
16        0.8
17        0.8
18        0.8
19        0.8
20        0.8
21        0.8
22        0.8
23        0.8
24        0.8
25        0.8
26        0.8
27        0.8
28        0.8
29          1
30          1
31          1
32          1
33          1
34          1
35          1
36          1
37          1
38          1
39          1
40          1
41          1
42          1
43          1
44          1
45          1
46          1
47          1
48    stepped
49    stepped
50    stepped
51    stepped
52    stepped
53    stepped
54    stepped
55    stepped
Name: C-rate, dtype: object

In [93]:
with open("./configs/crate_tests/paths_to_run_0.txt", 'r') as f:
    c = f.readlines()
c

['/usr/project/chembattery/chembattery_refac/configs/crate_tests/train_crate_test_03.yaml\n',
 '/usr/project/chembattery/chembattery_refac/configs/crate_tests/train_crate_test_05.yaml\n',
 '/usr/project/chembattery/chembattery_refac/configs/crate_tests/train_crate_test_step.yaml\n',
 '/usr/project/chembattery/chembattery_refac/configs/crate_tests/train_crate_test_step_pinn.yaml\n',
 '/usr/project/chembattery/chembattery_refac/configs/crate_tests/train_crate_test_05_pinn.yaml\n']

In [90]:
len(dataset.xCC)

9785838

In [76]:
dataset.masterList

cell_num
1        0.33
2        0.33
3        0.33
4        0.33
5         0.5
6         0.5
7         0.5
8         0.5
9         0.5
10        0.8
11        0.8
12        0.8
13        0.8
14        0.8
15        0.8
16        0.8
17        0.8
18        0.8
19        0.8
20        0.8
21        0.8
22        0.8
23        0.8
24        0.8
25        0.8
26        0.8
27        0.8
28        0.8
29          1
30          1
31          1
32          1
33          1
34          1
35          1
36          1
37          1
38          1
39          1
40          1
41          1
42          1
43          1
44          1
45          1
46          1
47          1
48    stepped
49    stepped
50    stepped
51    stepped
52    stepped
53    stepped
54    stepped
55    stepped
Name: C-rate, dtype: object

In [63]:
m = MLP(cfgr.model, cfgr.trainer)
print(m.trainer_config.epochs)
m.load_model_state("/usr/project/chembattery/chembattery_refac/runs/train_pinn_1e6/model")
print(m.trainer_config.epochs)

70
2


In [64]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]

In [68]:
[gpu.name.split(':')[-1] for gpu in tf.config.list_physical_devices('GPU')]

['0', '1', '2']

In [65]:
from gpu_utils import *
get_gpu_free_mem()

[23130, 23825, 23137]

In [71]:
for i in reversed(list(enumerate([1, 2, 3]))):
    print(i)

(2, 3)
(1, 2)
(0, 1)


In [56]:
i = 4
m_re = MLP(cfg1.model, cfg1.trainer)
m_fi = MLP(cfg2.model, cfg2.trainer)

m_re.load_model_state(f'/usr/project/chembattery/chembattery_refac/runs/reproduced_results_3/model/models/model_{i}')
m_fi.load_model_state(f'/usr/project/chembattery/chembattery_refac/runs/reproduced_results_3_finetune/model/models/model_{i}')

for i, (l_re, l_fi )in enumerate(zip(m_re.model.layers, m_fi.model.layers)):
    print(i)
    print([np.max(np.abs(l_re.weights[j]-l_fi.weights[j])) for j in range(len(l_re.weights))])


0
[0.0, 0.0]
1
[0.0, 0.0]
2
[0.0003064163, 0.00026588142]
3
[0.0010439306, 0.00032314658]


In [57]:
m_re.trainer_config.epochs

70

In [60]:
m_re.load_model_state("./runs/crate_control/model")

[autoreload of Model failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/usr/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 848, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/project/chembattery/chembattery_refac/Model.py", line 9, in <module>
    from constants import *
  File "/usr/project/chembattery/chembattery_refac/constants.py", line 13, in <module>
    "MLP": MLP
NameError: name 'MLP' is not defined
]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1649146/3639377093.py", line 1, in <module>
    m_re.load_model_state("./runs/crate_control/model")
  File "/usr/project/chembattery/chembattery_refac/Model.py", line 131, in load_model_state
    self.optimizer.apply_gradients(
  File "/usr/local/lib/python3.8/dist-packages/keras/src/optimizers/legacy/optimizer_v2.py", line 1323, in set_weights
    raise ValueError(
ValueError: You called `set_weights(weights)` on optimizer SGD with a  weight list of length 17, but the optimizer was expecting 1 weights. Provided weights: [31835, array([[ 1.42179624e-06,  8.36803974e-06, ...

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2105, in s

In [115]:
for value in X_train.index.get_level_values("cell_num").unique():
    if value in test_split:
        print(value)

In [111]:
dataset.xCC

Energy   Capacity  Voltage-8    Energy-8   Power-8
cell_num cycle                                                        
1        2       17.887097  11.096774   3.301269   17.758065  0.003499
         2       17.903226  11.096774   3.301331   17.774194  0.003499
         2       17.919355  11.096774   3.301406   17.790323  0.003499
         2       17.935484  11.129032   3.301425   17.806452  0.003500
         2       17.951613  11.129032   3.301505   17.822581  0.003500
...                    ...        ...        ...         ...       ...
55       21     128.387097  74.129032   3.596687  128.177419  0.005719
         21     128.419355  74.161290   3.596916  128.209677  0.005719
         21     128.435484  74.161290   3.597026  128.241935  0.005719
         21     128.467742  74.161290   3.597262  128.258065  0.005720
         21     128.483871  74.193548   3.597437  128.290323  0.005720

[9785838 rows x 5 columns]

In [116]:
model = MLP(cfg)
model.load_model_state("/usr/project/chembattery/chembattery_refac/runs/reproduced_results_1/model/models/model_0")

2025-07-02 20:16:57.223218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 692 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2025-07-02 20:16:57.223763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22289 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:67:00.0, compute capability: 8.6
2025-07-02 20:16:57.224166: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 21669 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [119]:
X_train, X_test, y_train, y_test, scaler = dataset.get_scaled_split(test_split, split_on)
pred_y = model.predict(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


   74/67568 [..............................] - ETA: 2:21

2025-07-02 20:17:46.738878: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


67568/67568 [==============================] - 142s 2ms/step


In [120]:
np.sum((pred_y-y_test)**2)**0.5

21902.317066767453

In [13]:
from eval_utils import compute_metrics, print_metrics

In [10]:
preds = []
for i in range(5):
    model = MLP(cfg)
    model.load_model_state(f"/usr/project/chembattery/chembattery_refac/runs/reproduced_results_1/model/models/model_{i}")
    X_train, X_test, y_train, y_test, scaler = dataset.get_scaled_split(test_split, split_on)
    y_pred = model.predict(X_test)
    metrics = compute_metrics(y_pred, y_test)
    print(i)
    print_metrics(metrics)
    print()
    preds.append(y_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
0

    1/67568 [..............................] - ETA: 1:11:41

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 132s 2ms/step
1

    1/67568 [..............................] - ETA: 1:13:14

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
2



/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
3

    1/67568 [..............................] - ETA: 1:10:33

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
4



In [17]:
ensemble_preds = np.array(preds).reshape(5, -1)
ensemble_preds = np.mean(ensemble_preds, axis=0)
metrics = compute_metrics(ensemble_preds, y_test)
print(i)
print(print_metrics(metrics))
print()

4

        "MAE": 0.026895336614543245,
        "Mean Error": -0.011644831910466467,
        "RMSE": 0.03878666672794386,
        "% RMSE": 51.01834028644382,
        "RMSPE": 33716.723830208764,
        "R²": 0.6824957795726763
    



In [15]:
for i,y_pred in enumerate(preds):
    metrics = compute_metrics(y_pred, y_test)
    print(i)
    print(print_metrics(metrics))
    print()

0

        "MAE": 0.031669625880544824,
        "Mean Error": -0.013317887149084884,
        "RMSE": 0.04535128942980448,
        "% RMSE": 59.65316722851638,
        "RMSPE": 43967.90471602293,
        "R²": 0.6428594708234547
    

1

        "MAE": 0.025596248099476877,
        "Mean Error": 0.0027670405088001185,
        "RMSE": 0.037799170309935466,
        "% RMSE": 49.71942928078887,
        "RMSPE": 36666.51100762615,
        "R²": 0.7304664451409433
    

2

        "MAE": 0.04462454734717367,
        "Mean Error": -0.02826921760082949,
        "RMSE": 0.07923569669269127,
        "% RMSE": 104.22328283726358,
        "RMSPE": 44142.33064643632,
        "R²": 0.31143946044485815
    

3

        "MAE": 0.02402684430083716,
        "Mean Error": 0.010083933262275123,
        "RMSE": 0.03474182419074962,
        "% RMSE": 45.69792555694128,
        "RMSPE": 33584.99183263324,
        "R²": 0.7134036763069967
    

4

        "MAE": 0.04245264589170171,
        "Mean Error": -0.0

In [ ]:
np.mean(np.array([
    [1, 2, 3],
    [2, 3, 4]
]), axis=0)

In [123]:
from sklearn.metrics import mean_squared_error

mean_squared_error(pred_y, y_test, squared=False)

61.69414919269864

In [125]:
print(np.sum((pred_y-y_test)**2/len(pred_y))**0.5)

61.69414919269863


In [128]:
s = RobustScaler()
a = np.random.rand(100).reshape(1, -1)
s.fit(a)
b = s.transform(a)
c = s.transform(b)
print(np.max(np.abs(a-b)))

0.9958682421754749


In [ ]:
print(np.max(np.abs(c-b)))

In [129]:
[1]*10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]